In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
np.set_printoptions(precision=4)

In [ ]:
data = pd.read_csv('final_df.csv')

In [ ]:
data.columns

Index(['season', 'round', 'weather_warm', 'weather_cold', 'weather_dry',
       'weather_wet', 'weather_cloudy', 'driver', 'grid', 'podium',
       'driver_points', 'driver_wins', 'driver_standings_pos',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'qualifying_time', 'driver_age', 'circuit_id_BAK',
       'circuit_id_adelaide', 'circuit_id_albert_park', 'circuit_id_americas',
       'circuit_id_bahrain', 'circuit_id_brands_hatch', 'circuit_id_catalunya',
       'circuit_id_detroit', 'circuit_id_estoril', 'circuit_id_galvez',
       'circuit_id_hockenheimring', 'circuit_id_hungaroring',
       'circuit_id_imola', 'circuit_id_indianapolis', 'circuit_id_interlagos',
       'circuit_id_istanbul', 'circuit_id_jacarepagua', 'circuit_id_jerez',
       'circuit_id_kyalami', 'circuit_id_magny_cours', 'circuit_id_marina_bay',
       'circuit_id_monaco', 'circuit_id_monza', 'circuit_id_nurburgring',
       'circuit_id_osterreichring', 'circuit_id_phoenix',


In [ ]:
datanew= data[['driver','qualifying_time','constructor_standings_pos','round','weather_warm','weather_cold','weather_dry','weather_wet','weather_cloudy','podium']]

In [ ]:
datanew.head()

,driver,qualifying_time,constructor_standings_pos,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,podium
0,keke_rosberg,0.000,0,1,False,False,True,False,False,15
1,prost,0.146,0,1,False,False,True,False,False,6
2,tambay,0.232,0,1,False,False,True,False,False,4
3,piquet,0.588,0,1,False,False,True,False,False,1
4,warwick,0.680,0,1,False,False,True,False,False,7


In [ ]:
data.head()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age,circuit_id_BAK,circuit_id_adelaide,circuit_id_albert_park,circuit_id_americas,circuit_id_bahrain,circuit_id_brands_hatch,circuit_id_catalunya,circuit_id_detroit,circuit_id_estoril,circuit_id_galvez,circuit_id_hockenheimring,circuit_id_hungaroring,circuit_id_imola,circuit_id_indianapolis,circuit_id_interlagos,circuit_id_istanbul,circuit_id_jacarepagua,circuit_id_jerez,circuit_id_kyalami,circuit_id_magny_cours,circuit_id_marina_bay,circuit_id_monaco,...,nationality_Austrian,nationality_Belgian,nationality_Brazilian,nationality_British,nationality_Canadian,nationality_Dutch,nationality_Finnish,nationality_French,nationality_German,nationality_Italian,nationality_Japanese,nationality_Mexican,nationality_Russian,nationality_Spanish,nationality_Swedish,constructor_arrows,constructor_bar,constructor_benetton,constructor_brabham,constructor_ferrari,constructor_footwork,constructor_force_india,constructor_haas,constructor_jaguar,constructor_jordan,constructor_larrousse,constructor_ligier,constructor_lotus_f1,constructor_mclaren,constructor_mercedes,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,0,0,0,0,0,0,0.000,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,0,0,0,0,0,0,0.146,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,0,0,0,0,0,0,0.232,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,0,0,0,0,0,0,0.588,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,0,0,0,0,0,0,0.680,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Classification

In [ ]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)


In [ ]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score
    return prediction_df

In [ ]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

### Logistic Regression

In [ ]:
# Random Forest Classifier

params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestClassifier(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('random_forest_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)


In [ ]:
df = data.copy()

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [ ]:
def score_regression(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [ ]:
params={'criterion': ['mse'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion = criterion,
                                          max_features = max_features, max_depth = max_depth, random_state = 1)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

In [ ]:
pd.DataFrame(comparison_dict).groupby('model')['score'].max()

model
random_forest_classifier    0.52381
random_forest_regressor     0.52381
Name: score, dtype: float64

In [ ]:
prediction_df.head()

,proba_0,proba_1,actual,predicted
0,0.35,0.65,1,1
1,0.72,0.28,2,0
2,0.93,0.07,4,0
3,0.94,0.06,6,0
4,0.97,0.03,3,0


In [ ]:
prediction_df.tail()

,proba_0,proba_1,actual,predicted
15,1.0,0.0,9,0
16,1.0,0.0,7,0
17,1.0,0.0,12,0
18,1.0,0.0,8,0
19,1.0,0.0,18,0
